<a href="https://colab.research.google.com/github/sajidcsecu/radioGenomic/blob/main/UnetinGPU_(3D).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# (1) Import Required Libraries

In [2]:
!pip install SimpleITK
!pip install pydicom===2.4.3
!pip install pydicom-seg
!pip install numpy==1.23.5
!pip install medpy

  Using cached medpy-0.5.2.tar.gz (156 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 971.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 87.5 MB/s eta 0:00:00
  Created wheel for medpy: filename=MedPy-0.5.2-py3-none-any.whl size=224710 sha256=56e15cef93b7a0ecb596adb36b1794f381d0d87f9b0644b2b83f0f89c9feced8
  Stored in directory: /root/.cache/pip/wheels/d4/33/ed/aaac5a347fb8d41679ca515b8f5c49dfdf49be15bdbb9a905d
Successfully built medpy
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydicom-seg 0.4.1 requires numpy<2.0.0,>=1.18.0, but you have numpy 2.2.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but

# (2) Import required Libraries

In [3]:
from sklearn.metrics import (
    jaccard_score,
    f1_score,
    recall_score,
    precision_score,
    accuracy_score,
)
import h5py
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
import torch.nn.functional as F
import random
import time
import csv
import numpy as np
import torch
import torch.nn as nn
import torch.optim.lr_scheduler as lr_scheduler
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
from typing import List
import torch.multiprocessing as mp
import h5py
from google.colab import drive
import torch.amp as amp
import cv2
from typing import List
from medpy.metric.binary import dc  # Dice coefficient for binary masks

# (3) Mount Google Drive

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


# (4) Data Preperation

In [5]:
class HDF5SegmentationDataset3D(Dataset):
    def __init__(self, hdf5_path, patch_size=(64, 512, 512), transform=None, return_pid=False):
        self.hdf5_path = hdf5_path
        self.patch_size = patch_size  # Desired output shape: (D, H, W)
        self.return_pid = return_pid
        self.transform = transform

        with h5py.File(self.hdf5_path, 'r') as f:
            self.patient_ids = list(f.keys())

    def pad_volume(self, volume):
        """Pads volume to match self.patch_size (D, H, W)."""
        c, d, h, w = volume.shape
        pd, ph, pw = self.patch_size
        pad_d = max(0, pd - d)
        pad_h = max(0, ph - h)
        pad_w = max(0, pw - w)
        padding = (0, pad_w, 0, pad_h, 0, pad_d)  # (w_left, w_right, h_top, h_bottom, d_front, d_back)
        return F.pad(volume, padding)

    def __len__(self):
        return len(self.patient_ids)

    def __getitem__(self, idx):
        pid = self.patient_ids[idx]
        with h5py.File(self.hdf5_path, 'r') as f:
            ct = f[pid]['ct'][:]
            mask = f[pid]['mask'][:]

        # Normalize CT to [0, 1]
        ct = (ct - np.min(ct)) / (np.max(ct) - np.min(ct) + 1e-8)

        # Convert to tensors and add channel dim: (1, D, H, W)
        ct = torch.tensor(ct, dtype=torch.float32).unsqueeze(0)
        mask = torch.tensor(mask, dtype=torch.float32).unsqueeze(0)

        # Pad to fixed size
        ct = self.pad_volume(ct)
        mask = self.pad_volume(mask)

        if self.transform:
            ct, mask = self.transform(ct, mask)

        if self.return_pid:
            return ct, mask, pid
        return ct, mask


# if __name__ == "__main__":
#     # Define the target directory
#     target_dir = "/content/drive/MyDrive/PhDwork/Segmentation"
#     # Change to the target directory
#     os.chdir(target_dir)
#     # Verify the change
#     print("Current Directory:", os.getcwd())

#     # Define device
#     device = "cuda" if torch.cuda.is_available() else "cpu"
#     print(f"Using device: {device}")

# #     # Define DATASET_DIR
#     DATASET_DIR = "./datasets"
# #   os.makedirs(DATASET_DIR, exist_ok=True) #make the directory if it does not exist.

#     batch_size = 8
#     num_workers = 0

#     # Dataset paths
#     train_path = os.path.join(DATASET_DIR, "train_dataset.hdf5")
#     valid_path = os.path.join(DATASET_DIR, "valid_dataset.hdf5")
#     test_path = os.path.join(DATASET_DIR, "test_dataset.hdf5")
#     train_dataset = HDF5SegmentationDataset3D(train_path)
#     train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers,pin_memory=True)

#     valid_dataset = HDF5SegmentationDataset3D(valid_path)
#     valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers,pin_memory=True)

#     test_dataset = HDF5SegmentationDataset3D(test_path)
#     test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers,pin_memory=True)

#     # Check a batch
#     for i, (images, masks) in enumerate(train_loader):
#         print(f"Batch {i}: Images shape = {images.shape}, Masks shape = {masks.shape}")

# 2. Unet

In [6]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv3d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm3d(out_channels),
            nn.LeakyReLU(inplace=True),
            nn.Conv3d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm3d(out_channels),
            nn.LeakyReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)

class UpSample(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.up = nn.ConvTranspose3d(in_channels, out_channels, kernel_size=2, stride=2)

    def forward(self, x):
        return self.up(x)

class EncoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels, dropout=0.1):
        super().__init__()
        self.conv = DoubleConv(in_channels, out_channels)
        self.pool = nn.MaxPool3d(2)
        self.dropout = nn.Dropout3d(p=dropout)

    def forward(self, x):
        x = self.conv(x)
        p = self.pool(x)
        return x, self.dropout(p)

class DecoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels, dropout=0.1):
        super().__init__()
        self.up = UpSample(in_channels, out_channels)
        self.conv = DoubleConv(out_channels * 2, out_channels)
        self.dropout = nn.Dropout3d(p=dropout)

    def forward(self, x, skip):
        x = self.up(x)
        x = torch.cat([x, skip], dim=1)
        x = self.conv(x)
        return self.dropout(x)

class UNet3D(nn.Module):
    def __init__(self, in_channels, out_channels, dropout=0.5):
        super().__init__()
        self.e1 = EncoderBlock(in_channels, 64, dropout=dropout)
        self.e2 = EncoderBlock(64, 128, dropout=dropout)
        self.e3 = EncoderBlock(128, 256, dropout=dropout)
        self.e4 = EncoderBlock(256, 512, dropout=dropout)

        self.b = DoubleConv(512, 1024)
        self.dropout_bottleneck = nn.Dropout3d(p=dropout)

        self.d1 = DecoderBlock(1024, 512, dropout=dropout)
        self.d2 = DecoderBlock(512, 256, dropout=dropout)
        self.d3 = DecoderBlock(256, 128, dropout=dropout)
        self.d4 = DecoderBlock(128, 64, dropout=dropout)

        self.outputs = nn.Conv3d(64, out_channels, kernel_size=1)

    def forward(self, x):
        s1, p1 = self.e1(x)
        s2, p2 = self.e2(p1)
        s3, p3 = self.e3(p2)
        s4, p4 = self.e4(p3)

        b = self.b(p4)
        b = self.dropout_bottleneck(b)

        d1 = self.d1(b, s4)
        d2 = self.d2(d1, s3)
        d3 = self.d3(d2, s2)
        d4 = self.d4(d3, s1)

        return self.outputs(d4)

# if __name__ == "__main__":
#     device = "cuda" if torch.cuda.is_available() else "cpu"
#     # 3D volume input: (batch_size, channels, depth, height, width)
#     input_image = torch.randn((1, 1, 64, 128, 128), dtype=torch.float32).to(device)
#     model = UNet(1, 1).to(device)
#     out = model(input_image)
#     print(out.shape)  # Expected output shape: (1, 1, 64, 128, 128) (depends on input)
#     print(device)
#     print(torch.cuda.is_available())

## 2. Loss Function

In [7]:
class DiceLoss(nn.Module):
    def __init__(self, smooth=1e-6, epsilon=1e-8):
        super(DiceLoss, self).__init__()
        self.smooth = smooth
        self.epsilon = epsilon

    def forward(self, preds, targets):
        preds = torch.sigmoid(preds)  # Apply sigmoid for binary segmentation
        # Flatten batch-wise
        preds = preds.contiguous().view(preds.size(0), -1)
        targets = targets.contiguous().view(targets.size(0), -1)

        intersection = (preds * targets).sum(dim=1)
        dice = (2. * intersection + self.smooth) / (
            preds.sum(dim=1) + targets.sum(dim=1) + self.smooth + self.epsilon
        )
        dice_loss = 1 - dice.mean()
        return dice_loss


class DiceBCELoss3D(nn.Module):
    def __init__(self, smooth=1e-6, epsilon=1e-8):
        super(DiceBCELoss3D, self).__init__()  # ✅ Correct class name here
        self.smooth = smooth
        self.epsilon = epsilon
        self.bce = nn.BCEWithLogitsLoss()  # Includes sigmoid internally

    def forward(self, preds, targets):
        bce_loss = self.bce(preds, targets)

        preds = torch.sigmoid(preds)
        preds = preds.contiguous().view(preds.size(0), -1)
        targets = targets.contiguous().view(targets.size(0), -1)

        intersection = (preds * targets).sum(dim=1)
        dice = (2. * intersection + self.smooth) / (
            preds.sum(dim=1) + targets.sum(dim=1) + self.smooth + self.epsilon
        )
        dice_loss = 1 - dice.mean()

        return bce_loss + dice_loss

# 3. Test

In [8]:
class UnetTest3D:
    def __init__(self, model: torch.nn.Module, test_loader: torch.utils.data.DataLoader,
                 result_dir: str, device: torch.device):
        self.model = model.to(device)
        self.test_loader = test_loader
        self.result_dir = result_dir
        self.device = device

        os.makedirs(result_dir, exist_ok=True)
        self.pred_volume_dir = os.path.join(result_dir, "predictions_hdf5")
        self.visual_dir = os.path.join(result_dir, "visualizations")

        os.makedirs(self.pred_volume_dir, exist_ok=True)
        os.makedirs(self.visual_dir, exist_ok=True)

    def calculate_metrics_3d(self, y_true: torch.Tensor, y_pred: torch.Tensor) -> List[float]:
        y_true = y_true.detach().cpu().numpy().astype(bool).reshape(-1)
        y_pred = (y_pred.detach().cpu().numpy() > 0.5).astype(bool).reshape(-1)

        score_jaccard = jaccard_score(y_true, y_pred, zero_division=1)
        score_f1 = f1_score(y_true, y_pred, zero_division=1)
        score_recall = recall_score(y_true, y_pred, zero_division=1)
        score_precision = precision_score(y_true, y_pred, zero_division=1)
        score_acc = accuracy_score(y_true, y_pred)

        return [score_jaccard, score_f1, score_recall, score_precision, score_acc]

    def save_visualization(self, volume: np.ndarray, mask: np.ndarray, pred: np.ndarray, sample_id: int) -> None:
        mid_slice = volume.shape[0] // 2
        img_slice = (volume[mid_slice] * 255).astype(np.uint8)
        mask_slice = (mask[mid_slice] > 0.5).astype(np.uint8) * 255
        pred_slice = (pred[mid_slice] > 0.5).astype(np.uint8) * 255

        h, w = img_slice.shape
        line = np.ones((h, 10), dtype=np.uint8) * 128
        cat = np.concatenate([img_slice, line, mask_slice, line, pred_slice], axis=1)

        file_path = os.path.join(self.visual_dir, f"sample_{sample_id}.png")
        cv2.imwrite(file_path, cat)
        print(f"✅ Visualization saved: {file_path}")

    def save_prediction_hdf5(self, pred_volume: np.ndarray, sample_id: int) -> None:
        file_path = os.path.join(self.pred_volume_dir, f"sample_{sample_id}.hdf5")
        with h5py.File(file_path, "w") as hf:
            hf.create_dataset("prediction", data=pred_volume.astype(np.uint8))
        print(f"📁 HDF5 prediction saved: {file_path}")

    def test(self):
        self.model.eval()
        print("🚀 Starting 3D U-Net testing...")

        overall_metrics = np.zeros(5)
        dice_scores = []
        times = []

        with torch.no_grad():
            for pid, (x, y) in enumerate(self.test_loader):
                x = x.to(self.device, dtype=torch.float32)  # [B, C, D, H, W]
                y = y.to(self.device, dtype=torch.float32)

                start = time.time()
                y_pred = torch.sigmoid(self.model(x))
                elapsed = time.time() - start
                times.append(elapsed)

                y_pred_bin = (y_pred > 0.5).float()

                for b in range(x.size(0)):
                    volume = x[b].squeeze().cpu().numpy()
                    true_mask = y[b].squeeze().cpu().numpy()
                    pred_mask = y_pred_bin[b].squeeze().cpu().numpy()

                    metrics = self.calculate_metrics(true_mask, pred_mask)
                    dice = dc(pred_mask.astype(bool), true_mask.astype(bool)) if np.any(true_mask) else 1.0

                    overall_metrics += np.array(metrics)
                    dice_scores.append(dice)

                    self.save_prediction_hdf5(pred_mask, sample_id=pid * x.size(0) + b)
                    self.save_visualization(volume, true_mask, pred_mask, sample_id=pid * x.size(0) + b)

        N = len(self.test_loader.dataset)
        avg_metrics = overall_metrics / N
        mean_dice = np.mean(dice_scores)
        fps = 1 / np.mean(times)

        print("\n📊 Average Test Metrics:")
        print(f"Jaccard:  {avg_metrics[0]:.4f}")
        print(f"F1 Score: {avg_metrics[1]:.4f}")
        print(f"Recall:   {avg_metrics[2]:.4f}")
        print(f"Precision:{avg_metrics[3]:.4f}")
        print(f"Accuracy: {avg_metrics[4]:.4f}")
        print(f"Dice Coef:{mean_dice:.4f}")
        print(f"FPS:      {fps:.2f}")

# 4. Training

In [11]:
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = float('inf')
        self.counter = 0

    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1

        if self.counter >= self.patience:
            print(f"⛔ Early stopping triggered after {self.patience} epochs without improvement!")
            return True
        return False


class UnetTrain3D:
    def __init__(self,
                 test_loader: torch.utils.data.DataLoader,
                 num_epochs: int,
                 lr: float,
                 train_loader: torch.utils.data.DataLoader,
                 valid_loader: torch.utils.data.DataLoader,
                 model_file: str,
                 loss_result_path: str,
                 device: torch.device,
                 result_dir: str,
                 visualize: bool = True,
                 vis_output_dir: str = None):

        self.test_loader = test_loader
        self.num_epochs = num_epochs
        self.lr = lr
        self.train_loader = train_loader
        self.valid_loader = valid_loader
        self.model_file = model_file
        self.loss_result_path = loss_result_path
        self.device = device
        self.result_dir = result_dir
        self.visualize = visualize
        self.vis_output_dir = vis_output_dir or os.path.join(result_dir, "visualizations")

        self.pred_volume_dir = os.path.join(self.result_dir, "predictions_hdf5")
        os.makedirs(self.result_dir, exist_ok=True)
        os.makedirs(self.pred_volume_dir, exist_ok=True)
        os.makedirs(self.vis_output_dir, exist_ok=True)

    def seeding(self, seed):
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True

    def epoch_time(self, start_time, end_time):
        elapsed_time = end_time - start_time
        elapsed_mins = int(elapsed_time / 60)
        elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
        return elapsed_mins, elapsed_secs

    def visualize_prediction_3d(self, input_volume, ground_truth, prediction, epoch=None, batch_idx=None):
        slice_idx = input_volume.shape[0] // 2
        ct_slice = input_volume[slice_idx].cpu().numpy()
        gt_slice = ground_truth[slice_idx].cpu().numpy()
        pred_slice = prediction[slice_idx].cpu().numpy()

        fig, axs = plt.subplots(1, 3, figsize=(12, 4))
        axs[0].imshow(ct_slice, cmap='gray')
        axs[0].set_title("CT Slice")
        axs[1].imshow(gt_slice, cmap='gray')
        axs[1].set_title("Ground Truth")
        axs[2].imshow(pred_slice, cmap='gray')
        axs[2].set_title("Prediction")
        for ax in axs:
            ax.axis("off")

        if epoch is not None and batch_idx is not None:
            plt.savefig(f"{self.vis_output_dir}/epoch{epoch}_batch{batch_idx}.png")
        plt.close()

    def train(self, model, loader, optimizer, loss_fn, device):
        scaler = amp.GradScaler()
        model.train()
        epoch_loss = 0.0

        for x, y in loader:
            x = x.to(device, dtype=torch.float32)
            y = y.to(device, dtype=torch.float32)

            optimizer.zero_grad()
            with amp.autocast(device_type=self.device.type):
                y_pred = model(x)
                loss = loss_fn(y_pred, y)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            epoch_loss += loss.item()

        return epoch_loss / len(loader)

    def evaluate(self, model, loader, loss_fn, device, epoch=0):
        model.eval()
        epoch_loss = 0.0
        with torch.no_grad():
            for batch_idx, (x, y) in enumerate(loader):
                x = x.to(device, dtype=torch.float32)
                y = y.to(device, dtype=torch.float32)
                y_pred = model(x)
                loss = loss_fn(y_pred, y)
                epoch_loss += loss.item()

                if self.visualize and batch_idx == 0:
                    y_pred_sigmoid = torch.sigmoid(y_pred)
                    y_pred_bin = (y_pred_sigmoid > 0.5).float()
                    self.visualize_prediction_3d(
                        input_volume=x[0, 0],
                        ground_truth=y[0, 0],
                        prediction=y_pred_bin[0, 0],
                        epoch=epoch,
                        batch_idx=batch_idx
                    )

        return epoch_loss / len(loader)

    def execute(self):
        model = UNet3D(in_channels=1, out_channels=1).to(self.device)
        optimizer = torch.optim.AdamW(model.parameters(), self.lr, weight_decay=1e-5)
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)
        loss_fn = DiceBCELoss3D()

        early_stopping = EarlyStopping(patience=10, min_delta=0.001)
        best_valid_loss = float("inf")
        results = {"train_loss": [], "valid_loss": []}
        torch.cuda.empty_cache()

        for epoch in range(self.num_epochs):
            start_time = time.time()

            train_loss = self.train(model, self.train_loader, optimizer, loss_fn, self.device)
            valid_loss = self.evaluate(model, self.valid_loader, loss_fn, self.device, epoch)

            if valid_loss < best_valid_loss:
                print(f"✅ Valid loss improved from {best_valid_loss:.4f} to {valid_loss:.4f}. Saving checkpoint.")
                best_valid_loss = valid_loss
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': best_valid_loss,
                }, self.model_file)

            end_time = time.time()
            mins, secs = self.epoch_time(start_time, end_time)
            results["train_loss"].append(train_loss)
            results["valid_loss"].append(valid_loss)
            print(f"Epoch {epoch+1}: ⏱️ {mins}m {secs}s | Train: {train_loss:.4f} | Val: {valid_loss:.4f}")

            if early_stopping(valid_loss):
                print("🛑 Early stopping triggered.")
                break
        torch.cuda.empty_cache()

        with open(self.loss_result_path, "w", newline="") as f:
            writer = csv.writer(f)
            for key, val in results.items():
                writer.writerow([key] + val)


In [12]:
def main():
    # Define the target directory
    target_dir = "/content/drive/MyDrive/PhDwork/Segmentation"
    # Change to the target directory
    os.chdir(target_dir)
    # Verify the change
    print("Current Directory:", os.getcwd())

    batch_size = 1   # Smaller batch size for 3D volumes due to memory
    num_epochs = 100
    lr = 1e-4

    output_dir = os.path.join(".", "Segmentation", "Results")
    os.makedirs(output_dir, exist_ok=True)
    loss_result_file = os.path.join(output_dir, "results_losses.csv")
    model_file = os.path.join(output_dir, "model.pth")
    test_result_path = os.path.join(output_dir, "results_test")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using Device:", device)

    DATASET_DIR = "./datasets"
    train_path = os.path.join(DATASET_DIR, "train_dataset.hdf5")
    valid_path = os.path.join(DATASET_DIR, "valid_dataset.hdf5")
    test_path = os.path.join(DATASET_DIR, "test_dataset.hdf5")

    print("Loading 3D Training Data...")
    train_dataset = HDF5SegmentationDataset3D(train_path)
    print("Loading 3D Validation Data...")
    valid_dataset = HDF5SegmentationDataset3D(valid_path)
    print("Loading 3D Test Data...")
    test_dataset = HDF5SegmentationDataset3D(test_path)

    num_workers = 0
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

    print(f"Train: {len(train_dataset)}, Valid: {len(valid_dataset)}, Test: {len(test_dataset)}")

    # ✅ Training
    trainer = UnetTrain3D(
        test_loader=test_loader,
        num_epochs=num_epochs,
        lr=lr,
        train_loader=train_loader,
        valid_loader=valid_loader,
        model_file=model_file,
        loss_result_path=loss_result_file,
        device=device,
        result_dir=output_dir,
        visualize=True,
        vis_output_dir=os.path.join(output_dir, "visualizations")
    )

    trainer.seeding(42)
    trainer.execute()

    # ✅ Testing
    model =UnetTrain3D(in_channels=1, out_channels=1).to(device)
    checkpoint = torch.load(model_file, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])

    utest = UnetTest3D(
    model=model,
    test_loader=test_loader,
    result_dir=test_result_path,
    device=device
    )
    utest.test()

if __name__ == "__main__":
    mp.set_start_method("spawn", force=True)  # Important for multiprocessing
    main()


Current Directory: /content/drive/MyDrive/PhDwork/Segmentation
Using Device: cuda
Loading 3D Training Data...
Loading 3D Validation Data...
Loading 3D Test Data...
Train: 340, Valid: 43, Test: 38


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/conv.py:720: UserWarning: cuDNN cannot be used for large non-batch-splittable convolutions if the V8 API is not enabled or before cuDNN version 9.3+. Consider upgrading cuDNN and/or enabling the V8 API for better efficiency. (Triggered internally at /pytorch/aten/src/ATen/native/Convolution.cpp:430.)
  return F.conv3d(


OutOfMemoryError: CUDA out of memory. Tried to allocate 108.00 GiB. GPU 0 has a total capacity of 39.56 GiB of which 15.44 GiB is free. Process 6006 has 24.10 GiB memory in use. Of the allocated memory 23.58 GiB is allocated by PyTorch, and 43.39 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)